In [2]:
!pip install docx2txt
!pip install sentence_transformers
!pip install PyPDF2

In [3]:
import os
import hashlib
from shutil import move

In [4]:
from sentence_transformers import SentenceTransformer
import PyPDF2
import docx2txt

In [5]:
# Define directory paths
document_folder = "C:\\Users\\817840\\OneDrive - Cognizant\\2024\\AI-Engineer-Roadmap-2024\\Project POC\\Finding duplicates and % match\\Files"
duplicates_folder = "C:\\Users\\817840\\OneDrive - Cognizant\\2024\\AI-Engineer-Roadmap-2024\\Project POC\\Finding duplicates and % match\\Files\\Duplicate_files"

In [6]:
def create_hash_database(folder_path):
    hash_database = {} #creating a dictionary for hash_database
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            try:
                with open(file_path, 'rb') as f:
                    md5_hash = hashlib.md5(f.read()).hexdigest()
                    sha256_hash = hashlib.sha256(f.read()).hexdigest()
                hash_database[filename] = (md5_hash, sha256_hash)
            except FileNotFoundError:
                print(f"Error: File '{filename}' not found. Hence skipping...")
    return hash_database   

In [7]:
def check_duplicate(filename, hash_database):
    if filename in hash_database:
        return True
    return False

In [9]:
def move_duplicate(filename, source_folder, destination_folder):
    source_path = os.path.join(source_folder, filename)
    destination_path = os.path.join(destination_folder, filename)
    try:
        move(source_path, destination_path)
        print(f"Duplicate '{filename}' moved to duplicate folder.")
    except Exception as e:
        print(f"Error moving '{filename}': {e}")

In [10]:
def extract_text(path):
    text = ""
    file_ext= path.split('.')[-1]    
    if file_ext == 'docx':
        text =docx2txt.process(path)
        
    elif file_ext == 'pdf':
        with open(path, "rb") as pdf_file:
            pdf_reader = PyPDF2.PdfReader(pdf_file)
            for page_num in range(len(pdf_reader.pages)):
                page = pdf_reader.pages[page_num]
                text += page.extract_text()
    else:
        pass

    return text

In [11]:
sentences_1=extract_text(r"C:\Users\817840\OneDrive - Cognizant\2024\Official\pythonProject\Finding duplicates and % match\Files\Machine learning - Wikipedia_1page.pdf")
sentences_2=extract_text(r"C:\Users\817840\OneDrive - Cognizant\2024\Official\pythonProject\Finding duplicates and % match\Files\sample.doc")

In [12]:
def main():
    #load existing hash database or create a new one
    if os.path.exists("hash_database.txt"):
        with open("hash_database.txt", "r") as f:
            hash_database = eval(f.read())
    else:
        hash_database = create_hash_database(document_folder)
        with open("hash_database.txt", "w") as f:
            f.write(str(hash_database))
    
    #Check for new documents
    for filename in os.listdir(document_folder):
        file_path = os.path.join(document_folder, filename)
        if os.path.isfile(file_path):
            #New document: update hash database and potentially move existing duplicates
            try:
                with open(file_path, "rb") as f:
                    md5_hash = hashlib.md5(f.read()).hexdigest()
                    sha256_hash = hashlib.sha256(f.read()).hexdigest()
                hash_database[filename] = (md5_hash, sha256_hash)
                
                #check for existing duplicates based on MD5 or SHA-256
                for existing_filename, existing_hashes in hash_database.items():
                    if(existing_filename != filename) and (md5_hash == existing_hashes[0]): #checking for MD5
                        #move existing duplicates
                        move_duplicate(existing_filename, document_folder, duplicates_folder)
                        break #only move one duplicate per new document
            except FileNotFoundError:
                print(f"Error: File'{filename}' not found. Skipping...")

    with open("hash_database.txt", "w") as f:
        f.write(str(hash_database))
    print("Hash database updated.")
    

if __name__ == "__main__":
    main()


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\817840\\OneDrive - Cognizant\\2024\\AI-Engineer-Roadmap-2024\\Project POC\\Finding duplicates and % match\\Files'

In [19]:
model = SentenceTransformer('BAAI/bge-large-zh-v1.5')
embeddings_1 = model.encode(sentences_1, normalize_embeddings=True)
embeddings_2 = model.encode(sentences_2, normalize_embeddings=True)
similarity = embeddings_1 @ embeddings_2.T
if similarity > 0.80:
    print(f"Similar Document with similarity score of {similarity}")
else:
    print(f"Different Document with similarity score of {similarity}")